In [7]:
#Cell 1: import packages
import numpy as np
import csv
from datetime import datetime
from dateutil.parser import parse
import binascii
import struct

In [8]:
#Cell 2: read in two tables：tab1 stores ad_exposures.csv, and tab2 stores sales_data.csv.
tab1 = []
tab2 = []
datafile1 = open('ad_exposures.csv', 'r')
reader1 = csv.reader(datafile1)
next(reader1)
for row in reader1:
    tab1.append(row)
datafile2 = open('sales_data.csv', 'r')
reader2 = csv.reader(datafile2)
next(reader2)
for row in reader2:
    tab2.append(row)
    
#Checking
expected_count1 = 10000
actual_count1 = len(tab1)
if expected_count1 != actual_count1:
    print(f"  Expected: {expected_count1}, Actual: {actual_count1} for tab1")
else:
    print(f" Read ad_exposures.csv correctly! There are {actual_count1} records!")
expected_count2 = 4492
actual_count2 = len(tab2)
if expected_count2 != actual_count2:
    print(f"  Expected: {expected_count2}, Actual: {actual_count2} for tab2")
else:
    print(f" Read sales_data.csv correctly! There are {actual_count2} records!")

 Read ad_exposures.csv correctly! There are 10000 records!
 Read sales_data.csv correctly! There are 4492 records!


In [9]:
#Cell 3: create two dictionaries:
#ad_exposures.csv is mapped to dic1. Each record is stored as key(user_id) + values([timestamp,creative_id]).
#sales_data.csv is mapped to dic2. Each record is stored as key(user_id) + values([timestamp,amount]).
dic1 = {}
dic2 = {}

for i in range(len(tab1)):
    user_id = tab1[i][0]
    timestamp = tab1[i][1]
    creative_id = tab1[i][2]
    if user_id not in dic1.keys():
        dic1[user_id] = [[timestamp,creative_id]]
    else:
        dic1[user_id].append([timestamp,creative_id])
for i in range(len(tab2)):
    user_id = tab2[i][0]
    timestamp = tab2[i][1]
    amount = tab2[i][2]
    if user_id not in dic2:
        dic2[user_id] = [[timestamp,amount]]
    else:
        dic2[user_id].append([timestamp,amount])

#Checking
count1 = 0
count2 = 0
for item in dic1.items():
    count1 +=len(dic1[item[0]])
for item in dic2.items():
    count2 +=len(dic2[item[0]])
if(count1!=expected_count1):
    print(f"  Expected: {expected_count1}, Actual: {count1} for dic1")
else:
    print(f" Create dic1 correctly! There are {count1} records!")
if(count2!=expected_count2):
    print(f"  Expected: {expected_count2}, Actual: {count2} for dic2")
else:
    print(f" Create dic2 correctly! There are {count2} records!")

 Create dic1 correctly! There are 10000 records!
 Create dic2 correctly! There are 4492 records!


In [10]:
#Cell 4: mapping money to creative_id

# First, we design two functions:
#Convert time format to hours
def to_hours(dt_time):
    dt_time = str(dt_time)
    if ',' not in dt_time:
        days = 0
        hours = int(dt_time.split(':')[0])
        minutes = int(dt_time.split(':')[1])
    else:
        first,second = dt_time.split(',')
        second = second.lstrip()
        days = int(first.split(' ')[0])
        hours = int(second.split(':')[0])
        minutes = int(second.split(':')[1])
    return 24*days+hours+minutes/60

#find the corresponding creative_id
def amount_to_id(user_id, buy_time, dic1):
    scan_time_slots = dic1[user_id]
    min_value = float('inf')
    status = False
    for ele in scan_time_slots:
        time = str(ele[0])
        if parse(time) < parse(buy_time):
            status = True
            diff = to_hours(parse(buy_time) - parse(time))
            if diff < min_value:
                min_value = diff
                id_value = ele[1]
    if status:
        return id_value
    else:
        return None

creative_id = {} # store format: key(creative_id)+values([amount,user_id])
for item2 in dic2.items():
    user_id = item2[0]
    buy_list = item2[1]
    for index, element in enumerate(buy_list):
        buy_time = str(element[0])
        amount = float(element[1])
        if user_id in dic1.keys():
            cur_id = amount_to_id(user_id, buy_time, dic1)
            if cur_id == None:
                continue
            elif cur_id not in creative_id:
                creative_id[cur_id] = [[amount,user_id]]
            else:
                creative_id[cur_id].append([amount,user_id])
#print(creative_id)

In [11]:
#Cell 5: Get the final result
final_creative = {} # store format: key(creative_id)+values([money,user_num,user_id])
for ele in creative_id.items():
    c_id = ele[0]
    info = ele[1]
    total_amount = 0
    user_id = []
    for index, element in enumerate(info):
        total_amount += element[0]
        if element[1] not in user_id:
            user_id.append(str(element[1]))
    final_creative[c_id] = [[total_amount,len(user_id),user_id]]  
#Calculate the total users
user_list = []
for ele in final_creative.items():
    user_id_list = ele[1][0][2]
    for x in user_id_list:
        if x not in user_list:
            user_list.append(x)
total_user_num = len(user_list) 
print(total_user_num)

1080


In [12]:
#Cell 6: Write the final result into csv
result = []
for ele in final_creative.items():
    r_c_id = ele[0]
    r_user_num = ele[1][0][1]
    r_amount = ele[1][0][0]
    result.append([r_c_id,r_user_num,r_amount])
dt = datetime.strptime(str(datetime.now()).split('.')[0], '%Y-%m-%d %H:%M:%S')
timestamp = dt.timestamp() # assume dt is a local time
datehex = str(binascii.hexlify(struct.pack('<I', round(timestamp))))[1:].replace("'","")
filename = f'Result_{datehex}.csv'
file = open(filename, 'a+', newline ='')
with file:
    csvwriter = csv.writer(file)
    csvwriter.writerows(result)
print(f">> Results are written to {filename}")

>> Results are written to Result_0f032163.csv
